In [16]:
from datetime import datetime, timedelta
import pandas as pd
import folium
from sklearn.cluster import DBSCAN
import numpy as np
import os


df_gu = pd.read_csv("../../data/서울시 광진구 안심이 CCTV 연계현황.csv", encoding='cp949')
df_gu

,자치구,안심 주소,위도,경도,CCTV 수량,수정 일시
0,광진구,공원-광장-045-01,37.5460,127.1039,1,2022-12-01
1,광진구,공원-광장-045-02,37.5460,127.1039,1,2022-12-01
2,광진구,공원-광장-045-03,37.5460,127.1039,1,2022-12-01
3,광진구,공원-광장-045-비상벨,37.5460,127.1039,1,2022-12-01
4,광진구,공원-광장-046-01,37.5482,127.1006,1,2022-12-01
...,...,...,...,...,...,...
4086,광진구,주차-화양-034-03,37.5421,127.0714,1,2022-12-01
4087,광진구,주차-화양-035-01,37.5409,127.0709,1,2022-12-01
4088,광진구,주차-화양-035-02,37.5409,127.0709,1,2022-12-01
4089,광진구,주차-화양-035-03,37.5409,127.0709,1,2022-12-01


In [17]:
# 파일이 있는 폴더 경로 (예: "./data/cctv" 또는 "." 그대로)
BASE_DIR = "../../data/서울시 광진구 안심이 CCTV 연계현황.csv"  # CSV 파일이 현재 디렉토리에 있다면 이렇게 설정

# 시작 날짜와 종료 날짜 (여러 날짜일 경우 loop, 단일 날짜면 동일하게)
START_DATE = "2022-12-01"
END_DATE = "2022-12-01"

start_dt = datetime.strptime(START_DATE, "%Y-%m-%d")
end_dt = datetime.strptime(END_DATE, "%Y-%m-%d")

In [26]:
import pandas as pd
import folium
from sklearn.cluster import DBSCAN
import numpy as np

# 📁 정확한 파일 경로 설정
file_path = "../../data/서울시 광진구 안심이 CCTV 연계현황.csv"  # 실제 파일 위치로 수정

try:
    # 1. 파일 읽기
    df = pd.read_csv(file_path, encoding='cp949')

    # 2. 전처리
    df[['시설유형', '구역코드', '세부번호']] = df['안심 주소'].str.extract(r'([가-힣\-]+)-(\d+)-?(.+)?')
    df['CCTV 수량'] = pd.to_numeric(df['CCTV 수량'], errors='coerce').fillna(0).astype(int)
    df['위도'] = pd.to_numeric(df['위도'], errors='coerce')
    df['경도'] = pd.to_numeric(df['경도'], errors='coerce')

    # 3. 클러스터링
    coords = df[['위도', '경도']].dropna().values
    kms_per_radian = 6371.0088
    epsilon = 0.1 / kms_per_radian  # 약 100m
    db = DBSCAN(eps=epsilon, min_samples=2, algorithm='ball_tree', metric='haversine').fit(np.radians(coords))
    df['cluster'] = db.labels_

    # 4. 지도 시각화
    m = folium.Map(location=[37.54, 127.08], zoom_start=13)
    for _, row in df.iterrows():
        folium.CircleMarker(
            location=[row['위도'], row['경도']],
            radius=row['CCTV 수량'] * 2,
            popup=f"{row['시설유형']} (cluster: {row['cluster']})",
            color='blue',
            fill=True
        ).add_to(m)

    # 결과 저장
    m.save("cctv_map.html")
    df.to_csv("processed_cctv.csv", index=False)
    print("✅ 분석 및 저장 완료!")

except Exception as e:
    print(f"❌ 에러 발생: {e}")


✅ 분석 및 저장 완료!


In [22]:
!ls ../../data

 README.md
'서울시 광진구 안심이 CCTV 연계현황.csv'
 서울특별시_신호등표준데이터_20240425.csv


In [27]:
import pandas as pd

# 정확한 경로
file_path = "../../data/서울시 광진구 안심이 CCTV 연계현황.csv"

# CSV 읽기 (cp949 인코딩 사용)
df = pd.read_csv(file_path, encoding="cp949")

# 상위 5행만 출력
df.head()

,자치구,안심 주소,위도,경도,CCTV 수량,수정 일시
0,광진구,공원-광장-045-01,37.5460,127.1039,1,2022-12-01
1,광진구,공원-광장-045-02,37.5460,127.1039,1,2022-12-01
2,광진구,공원-광장-045-03,37.5460,127.1039,1,2022-12-01
3,광진구,공원-광장-045-비상벨,37.5460,127.1039,1,2022-12-01
4,광진구,공원-광장-046-01,37.5482,127.1006,1,2022-12-01
